# P2 : Data wrangle OpenStreetMaps Data
## Data Wrangling with MongoDB

#### _HanByul Yang_

Map Area : part of Silicon Valley (including San José and Santa Clara), CA, United States 

https://www.openstreetmap.org/#map=13/37.3780/-121.9116
  * longitude -122.0554 ~ -121.8277
  * latitude 37.4588 ~ 37.2969

### 0. Preparing map files
#### 0-1. Creating map_sample.osm

Made an `.osm` file containing a sample part of the map region.
  * Used a given sample code in https://www.udacity.com/course/viewer#!/c-nd002/l-3168208620/m-3189488621
  * Executed `create_sample.py` for genereting `map_sample.osm` 

In [1]:
run create_sample.py

#### 0-2. Creating map.osm.json and Inserting into DB

Converted `map.osm` to `map.osm.json`
  * Used `data.py` from Lesson_6 12-Preparing_for_Database
  * Referenced Lesson_4 02-Inserting_into_DB for inserting python dictionary to mongoDB
  * Auditing and cleaning addresses while parsing `map.osm` 

In [2]:
run data.py

### 1. Problem encountered in the Map

#### 1-1 Abbreviated street names
After downloading map file(map.osm), I audited it with `audit.py` from "Lesson 6, 11-Improving Street Names". I noticed some street names were abbreviated with several ways, such that "Bollinger Rd", "wilcox ave" or "Monroe St".

`audit.py` generated `audit.log` which has patterns of address with and `clean.log` which shows cleaned name.

In [3]:
run audit.py

Here's cleaned result.

name | better name
------------- | ------------- 
E. Santa Clara St. | E. Santa Clara Street
Bollinger Rd | Bollinger Road
Homestead Rd | Homestead Road
wilcox ave | wilcox Avenue
E Santa Clara ave | E Santa Clara Avenue
Zanker Rd., San Jose, CA | Zanker Road., San Jose, CA
Monroe St | Monroe Street
170 S Market St | 170 S Market Street
Casa Verde St | Casa Verde Street
South Bascom Ave | South Bascom Avenue
Blake Ave | Blake Avenue
Park Ave | Park Avenue
Ann Arbor Ave | Ann Arbor Avenue
The Alameda Ave | The Alameda Avenue
E Santa Clara Ave | E Santa Clara Avenue
Seaboard Ave | Seaboard Avenue
Walsh Ave | Walsh Avenue
N Blaney Ave | N Blaney Avenue
Cabrillo Ave | Cabrillo Avenue

#### 1-2 Inserting map data to DB
At first, I tried to insert map data (python dictionary format) from "map.osm.json" file. But JSON parser of python can't load large file. So I decided to use generated python dictionary while running "data.py".

Here's code snippet.
```python
    map_file_name = 'map.osm'
    data = process_map(map_file_name, False)

    from pymongo import MongoClient
    client = MongoClient("mongodb://localhost:27017")
    db = client.p2
    db.osm.insert(data)
```

#### 1-3 Slowness of parsing osm file
I used `xml.etree.ElementTree` package for paring osm file at first and it took quite a lot time.  
So I changed with `xml.etree.cElementTree` for parsing osm file. 

For example, running `audit.py` changes from 21.4s to 4.8s after changing parsing package.

### 2. Data Overview

This section contains basic statistics about the dataset and MongoDB queries used to gather them.

#### File sizes
`map.osm ........` 77.7 MB  
`map.osm.json ...` 82.1 MB

In [4]:
# Prepare DB
from pymongo import MongoClient
from pprint import pprint

client = MongoClient("mongodb://localhost:27017")
db = client.p2

##### Number of documents

In [5]:
print db.osm.find().count()

373403


##### Number of nodes

In [6]:
print db.osm.find({"type" : "node"}).count()

326104


##### Number of ways

In [7]:
print db.osm.find({"type" : "way"}).count()

47287


##### Number of unique users

In [8]:
print len(db.osm.distinct("created.user"))

720


##### Top 10 contributing user

In [9]:
query = [
    {"$group" : {"_id" : "$created.user", "count" : {"$sum":1}}},
    {"$sort" : {"count" : -1}},
    {"$limit" : 10}
]
result = db.osm.aggregate(query)['result']
pprint(result)

[{u'_id': u'n76', u'count': 83824},
 {u'_id': u'mk408', u'count': 43243},
 {u'_id': u'Bike Mapper', u'count': 40390},
 {u'_id': u'matthieun', u'count': 27621},
 {u'_id': u'nmixter', u'count': 20992},
 {u'_id': u'KindredCoda', u'count': 12745},
 {u'_id': u'StellanL', u'count': 8291},
 {u'_id': u'woodpeck_fixbot', u'count': 7493},
 {u'_id': u'adbrown', u'count': 7167},
 {u'_id': u'oldtopos', u'count': 6847}]


In [10]:
print (result[0]['count'])

83824


##### Number of users appearing only once (having 1 post)

In [11]:
query = [
    {"$group" : {"_id" : "$created.user", "count" : {"$sum":1}}},
    {"$group":{"_id":"$count", "num_users":{"$sum":1}}},
    {"$sort":{"_id":1}}, 
    {"$limit":1}
]
result = db.osm.aggregate(query)['result']
pprint(result)

[{u'_id': 1, u'num_users': 162}]


### 3. Additional Ideas

#### Contributor statistics  
The small number of users are contributed much of data but differences between top contributers are not big enough. There is an user ('woodpeck_fixbot') who seems to edit automatically. Here are some statistics.

In [12]:
query = [
    {"$group" : {"_id" : "$created.user", "count" : {"$sum":1}}},
    {"$sort" : {"count" : 1}},
]
user_cont_result = db.osm.aggregate(query)['result']

query = [
    {"$group" : {"_id" : "$created.user", "count" : {"$sum":1}}},
    {"$sort" : {"count" : -1}},
    {"$limit" : 10},
    {"$group" : {"_id" : 0, "total" : {"$sum" : "$count"}}}
]

top10Docs = (db.osm.aggregate(query)['result'][0]['total'])
totalDocs = db.osm.find().count()

topUserContribu = user_cont_result[-1]['count'] / float(totalDocs) * 100

one_pecent_docs = totalDocs * 0.01
user_1per = 0
doc_count = 0
for user_result in user_cont_result:
    if (doc_count > one_pecent_docs):
        break
    doc_count += user_result['count']
    user_1per += 1

print('Top user contribution ("{0}") - {1:.2f}%'.format(user_cont_result[-1]['_id'], topUserContribu))
print("Top 10 uesrs contributions    - {0:.2f}%".format((top10Docs / float(totalDocs) * 100)))
print('Combined number of users making up only 1% of posts - {0} (about {1:.0f}% of all users)'.format(user_1per, float(user_1per) / len(user_cont_result) * 100))

Top user contribution ("n76") - 22.45%
Top 10 uesrs contributions    - 69.26%
Combined number of users making up only 1% of posts - 509 (about 71% of all users)


#### Additional data exploration using MongoDB queries

##### Top 10 appearing amenities

In [13]:
query = [
    {"$match":{"amenity":{"$exists":1}}},
    {"$group":{"_id":"$amenity","count":{"$sum":1}}},
    {"$sort":{"count":-1}}, 
    {"$limit":10}
]
result = db.osm.aggregate(query)['result']
pprint(result)

[{u'_id': u'parking', u'count': 768},
 {u'_id': u'restaurant', u'count': 458},
 {u'_id': u'school', u'count': 322},
 {u'_id': u'fire_hydrant', u'count': 247},
 {u'_id': u'place_of_worship', u'count': 183},
 {u'_id': u'fast_food', u'count': 175},
 {u'_id': u'bicycle_parking', u'count': 149},
 {u'_id': u'fuel', u'count': 113},
 {u'_id': u'cafe', u'count': 113},
 {u'_id': u'toilets', u'count': 86}]


##### Details about Top 1 amenity : parking
There are 17 free parking lot.

In [14]:
query = [
    {"$match":{"amenity":{"$exists":1}, "amenity":"parking"}},
    {"$group":{"_id":"$fee", "count":{"$sum":1}}},
    {"$sort":{"count":-1}},
    {"$limit":5}
]
result = db.osm.aggregate(query)['result']
pprint(result)

[{u'_id': None, u'count': 744},
 {u'_id': u'no', u'count': 17},
 {u'_id': u'yes', u'count': 7}]


##### Details about Top 2 amenity : restaurant

In [15]:
query = [
    {"$match":{"amenity":{"$exists":1}, "amenity":"restaurant"}},
    {"$group":{"_id":"$cuisine", "count":{"$sum":1}}},
    {"$sort":{"count":-1}},
    {"$limit":10}
]
result = db.osm.aggregate(query)['result']
pprint(result)

[{u'_id': None, u'count': 168},
 {u'_id': u'chinese', u'count': 38},
 {u'_id': u'indian', u'count': 29},
 {u'_id': u'mexican', u'count': 26},
 {u'_id': u'pizza', u'count': 23},
 {u'_id': u'american', u'count': 17},
 {u'_id': u'japanese', u'count': 17},
 {u'_id': u'thai', u'count': 15},
 {u'_id': u'italian', u'count': 13},
 {u'_id': u'sandwich', u'count': 12}]


##### Biggeest religion

In [16]:
query = [
    {"$match":{"amenity":{"$exists":1}, "amenity":"place_of_worship"}},
    {"$group":{"_id":"$religion", "count":{"$sum":1}}},
    {"$sort":{"count":-1}}, {"$limit":1}
]
result = db.osm.aggregate(query)['result']
pprint(result)

[{u'_id': u'christian', u'count': 161}]


#### Conclusion
As the results, The top amenity of given map area is parking lot. But I couldn't analyze whether they are free or not because of missing tag value of 'fee'. I guess most of 'None' are require fee.  
Most popular cuisine is chinese in given map area and No. 2 is indian. I think the results shows cultural diversity of silicon valley.
